In [11]:
import pandas as pd
import numpy as np
import os
import json
import urllib.request
import math
import itertools

# 네이버 API 키 설정
NAVER_CLIENT_ID = 'uo2xxmprpi'
NAVER_CLIENT_SECRET = 'DGA3bnsb1NgcAUA5CJAjtwlksM8O1281OUcRNbpu'
CACHE_DIR = 'cache'  # 캐시 파일을 저장할 디렉터리

# 캐시 디렉터리 생성
if not os.path.exists(CACHE_DIR):
    os.makedirs(CACHE_DIR)

In [12]:
# API 호출을 수행하는 함수
def get_route_time(start, waypoints, goal, option='traoptimal'):
    """경로의 소요 시간을 API를 통해 계산"""
    waypoints_str = '|'.join([f"{lon},{lat}" for lat, lon in waypoints])
    url = f"https://naveropenapi.apigw.ntruss.com/map-direction/v1/driving?start={start[1]},{start[0]}&goal={goal[1]},{goal[0]}&waypoints={waypoints_str}&option={option}"
    
    request = urllib.request.Request(url)
    request.add_header('X-NCP-APIGW-API-KEY-ID', NAVER_CLIENT_ID)
    request.add_header('X-NCP-APIGW-API-KEY', NAVER_CLIENT_SECRET)
    
    try:
        response = urllib.request.urlopen(request)
        data = json.loads(response.read().decode('utf-8'))
        if 'route' in data and 'traoptimal' in data['route']:
            return data['route']['traoptimal'][0]['summary']['duration']  # 소요시간 반환 (밀리초 단위)
    except Exception as e:
        print(f"Error fetching route from {start} to {goal}: {e}")
    
    return float('inf')  # 실패 시 매우 큰 값 반환

def get_coordinates_from_address(address, retries=1):
    """주소 위경도 변환"""
    url = f"https://naveropenapi.apigw.ntruss.com/map-geocode/v2/geocode?query={urllib.parse.quote(address)}"
    
    request = urllib.request.Request(url)
    request.add_header('X-NCP-APIGW-API-KEY-ID', NAVER_CLIENT_ID)
    request.add_header('X-NCP-APIGW-API-KEY', NAVER_CLIENT_SECRET)
    
    attempt = 0
    while attempt <= retries:
        try:
            response = urllib.request.urlopen(request)
            if response.getcode() == 200:   #정상 작동 코드 값
                response_body = json.loads(response.read().decode('utf-8'))
                if 'addresses' in response_body and len(response_body['addresses']) > 0:
                    lat = float(response_body['addresses'][0]['y'])
                    lng = float(response_body['addresses'][0]['x'])
                    return lat, lng
                else:
                    print(f"Geocoding failed for address: {address} (no addresses found)")
            else:
                print(f"Geocoding failed for address: {address} (HTTP code {response.getcode()})")
        except Exception as e:
            print(f"Error: Unable to contact Naver Maps API for address: {address}. Exception: {e}")

        attempt += 1
        if attempt <= retries:
            print(f"Retrying ({attempt}/{retries})...")
    
    return None, None

def milliseconds_to_hms(milliseconds):
    """밀리초를 시:분:초 형식으로 변환"""
    seconds = milliseconds / 1000
    minutes = seconds / 60
    hours = minutes / 60
    return f"{int(hours)}h {int(minutes % 60)}m {int(seconds % 60)}s"

In [13]:
def haversine_distance(coord1, coord2):
    """두 지점 간의 하버사인 거리 계산"""
    R = 6371.0  # 지구 반경 (킬로미터)
    
    lat1, lon1 = coord1
    lat2, lon2 = coord2
    
    lat1_rad = math.radians(lat1)
    lon1_rad = math.radians(lon1)
    lat2_rad = math.radians(lat2)
    lon2_rad = math.radians(lon2)
    
    dlat = lat2_rad - lat1_rad
    dlon = lon2_rad - lon1_rad
    
    a = math.sin(dlat / 2) ** 2 + math.cos(lat1_rad) * math.cos(lat2_rad) * math.sin(dlon / 2) ** 2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    
    distance = R * c
    return distance

def find_closest_stores_to_seeker(start_location, store_locations, num_closest):
    """구직자와 직선 거리상 가까운 매장 검색"""
    distances = [(i, haversine_distance(start_location, location)) for i, location in enumerate(store_locations)]
    closest_stores = sorted(distances, key=lambda x: x[1])[:num_closest]
    return [store[0] for store in closest_stores]

In [14]:
def calculate_store_to_store(store_locations):
    """매장 간의 거리 행렬을 캐시에서 로드하거나 계산"""
    print("Calculating store-to-store distance matrix...")
    cache_file_path = os.path.join(CACHE_DIR, 'store_distance_matrix.npy')
    distance_matrix = load_distance_matrix_from_cache(cache_file_path)
    
    if distance_matrix is None:
        num_stores = len(store_locations)
        distance_matrix = np.zeros((num_stores, num_stores))
        
        for i, j in itertools.product(range(num_stores), repeat=2):
            if i != j:
                distance_matrix[i, j] = get_route_time(store_locations[i], [], store_locations[j])
                print(f"Distance from store {i} to store {j}: {distance_matrix[i, j]}")
        
        save_distance_matrix_to_cache(distance_matrix, cache_file_path)
    
    return distance_matrix

def load_distance_matrix_from_cache(cache_file):
    """저장된 캐시 파일에서 매장 간 거리 행렬 로드"""
    print(f"Attempting to load distance matrix from cache at {cache_file}...")
    if os.path.exists(cache_file):
        try:
            distance_matrix = np.load(cache_file)
            print("Loaded distance matrix from cache.")
            return distance_matrix
        except Exception as e:
            print(f"Error loading cached distance matrix: {e}")
    return None

def save_distance_matrix_to_cache(distance_matrix, cache_file):
    """매장 간 거리 행렬을 캐시 파일에 저장"""
    try:
        np.save(cache_file, distance_matrix)
        print(f"Saved distance matrix to cache at {cache_file}.")
    except Exception as e:
        print(f"Error saving distance matrix to cache: {e}")

In [15]:
def calculate_route_times(route, time_matrix):
    """경로에 대한 총 소요 시간을 계산."""
    return sum(time_matrix[route[i]][route[i + 1]] for i in range(len(route) - 1))

In [16]:
def cache_seeker_times(start_location, store_locations, is_return=False):
    """구직자 위치에서 각 매장으로 가는 시간, 돌아오는 시간을 캐시"""
    direction = 'end' if is_return else 'start'
    cache_file = os.path.join(CACHE_DIR, f'{direction}_times_cache_{hash(start_location)}.npy')
    print(f"Caching seeker times to {cache_file} (is_return={is_return})...")
    
    if os.path.exists(cache_file):
        print(f"Loading cached seeker times from {cache_file}...")
        return np.load(cache_file)
    
    num_closest_stores = 20
    closest_stores_indices = find_closest_stores_to_seeker(start_location, store_locations, num_closest=num_closest_stores)
    print(f"Closest stores indices: {closest_stores_indices}")
    closest_store_locations = [store_locations[i] for i in closest_stores_indices]
    
    times = np.full(len(store_locations), float('inf'))  # 전체 매장 수 만큼 inf로 초기화
    
    for i, location in enumerate(closest_store_locations):
        if is_return:
            times[closest_stores_indices[i]] = get_route_time(location, [], start_location)
        else:
            times[closest_stores_indices[i]] = get_route_time(start_location, [], location)
        print(f"Time to/from store {closest_stores_indices[i]}: {times[closest_stores_indices[i]]}")
    
    np.save(cache_file, times)
    return times

def load_times_seeker_cache(start_location, store_locations, is_return=False):
    """저장된 캐시 파일에서 데이터를 로드하거나, 없으면 새로 생성"""
    direction = 'end' if is_return else 'start'
    cache_file = os.path.join(CACHE_DIR, f'{direction}_times_cache_{hash(start_location)}.npy')
    print(f"Loading times seeker cache from {cache_file} (is_return={is_return})...")
    
    if os.path.exists(cache_file):
        print(f"Loaded seeker times from cache.")
        return np.load(cache_file)
    else:
        return cache_seeker_times(start_location, store_locations, is_return)
    
def delete_cache_files(start_location):
    """구직자 위치에 대한 start 및 end 캐시 파일 삭제"""
    for is_return in [False, True]:
        direction = 'end' if is_return else 'start'
        cache_file = os.path.join(CACHE_DIR, f'{direction}_times_cache_{hash(start_location)}.npy')
        
        if os.path.exists(cache_file):
            os.remove(cache_file)
            print(f"Deleted cache file: {cache_file}")
        else:
            print(f"No cache file found at: {cache_file}")

In [17]:
def calculate_route_times(route, time_matrix):
    """경로에 대한 총 소요 시간 계산."""
    return sum(time_matrix[route[i]][route[i + 1]] for i in range(len(route) - 1))

def find_top_routes(start_location, store_locations, store_names, num_routes=10):
    """최적의 루트 탐색"""
    print(f"Finding top routes for job seeker at {start_location}...")
    num_stores = len(store_locations)
    best_routes = []

    print(f"Number of stores: {num_stores}")

    print(f"Finding top routes for job seeker at {start_location}...")
    
    # 전체 매장 간 거리 행렬 계산
    store_to_store_distance_matrix = calculate_store_to_store(store_locations)
    print(f"Store-to-store distance matrix calculated:\n{store_to_store_distance_matrix}")

    # 구직자 위치에서 가까운 매장 찾기
    closest_stores_indices = find_closest_stores_to_seeker(start_location, store_locations, num_closest=20)
    print(f"Closest stores to job seeker: {closest_stores_indices}")

    # 구직자 위치에서 각 매장으로 가는 시간과 돌아오는 시간을 캐시에서 로드 (캐시가 없는 경우 생성)
    start_times = load_times_seeker_cache(start_location, store_locations, is_return=False)
    end_times = load_times_seeker_cache(start_location, store_locations, is_return=True)
    print(f"Start times: {start_times}")
    print(f"End times: {end_times}")

    # 전체 매장 간 조합 평가
    print("Evaluating all combinations of stores from all stores...")
    for route_indices in itertools.combinations(closest_stores_indices, 5):
        route = list(route_indices)
        print(f"Evaluating route: {route}")
        
        # 구직자 위치에서 첫 매장으로 가는 시간
        start_time = start_times[route[0]]
        if start_time == float('inf'):
            print(f"Skipping route {route} due to infinite start time.")
            continue
        
        # 매장 간의 소요 시간 (캐시된 시간 행렬 사용)
        route_time = start_time + calculate_route_times(route, store_to_store_distance_matrix)
        
        # 마지막 매장에서 구직자 위치로 돌아오는 시간
        end_time = end_times[route[-1]]
        if end_time == float('inf'):
            print(f"Skipping route {route} due to infinite end time.")
            continue
        route_time += end_time
        
        # 최적 경로 리스트에 추가
        best_routes.append((route_time, route))
        print(f"Route {route} added with total time {route_time}")
    
    # 경로를 총 소요 시간 기준으로 정렬
    best_routes.sort(key=lambda x: x[0])
    print(f"Best routes sorted by time: {best_routes}")
    
    # 최적 경로 출력
    top_routes = best_routes[:num_routes]
    print(f"Top {num_routes} routes:")
    for route_time, route in top_routes:
        route_names = ['Start'] + [store_names[i] for i in route] + ['Start']
        route_time_str = milliseconds_to_hms(route_time)
        print(f"Route: {' -> '.join(route_names)}, Time: {route_time_str}")
    
    return top_routes

In [18]:
def save_top_routes_to_file(top_routes, store_locations, store_names, job_seeker_location, filename='top_routes.json'):
    """
    경로 데이터를 JSON 파일에 저장
    
    :param top_routes: 최상위 경로 리스트 (경로 시간, 경로 인덱스의 튜플)
    :param store_locations: 매장 위치 리스트 (위도, 경도)
    :param store_names: 매장 이름 리스트
    :param job_seeker_location: 구직자 위치 (위도, 경도)
    :param filename: 저장할 JSON 파일의 이름
    """
    routes_data = []
    print(f"Job Seeker Location: {job_seeker_location}")
    
    try:
        # job_seeker_location을 float으로 변환 (만약 numpy.float64인 경우)
        job_seeker_location = (float(job_seeker_location[0]), float(job_seeker_location[1]))
        print(f"Job Seeker Location: {job_seeker_location}")
    except ValueError as e:
        print(f"Error converting job_seeker_location to float: {e}")
        return  # 에러 발생 시 함수 종료
    
    for index, (route_time, route) in enumerate(top_routes):
        # 경로 이름을 고유하게 지정
        route_name = f'Route {index + 1}'
        
        # 경로 좌표 및 매장 이름을 포함한 리스트 생성
        coordinates = [
            {"name": "현재위치", "coords": list(map(float, job_seeker_location))}
        ]
        coordinates += [
            {"name": store_names[i], "coords": list(map(float, store_locations[i]))} for i in route
        ]
        coordinates.append(
            {"name": "현재위치", "coords": list(map(float, job_seeker_location))}
        )
        
        # 실제 소요 시간 측정
        actual_time = get_route_time(job_seeker_location, [store_locations[i] for i in route], job_seeker_location)
        
        route_data = {
            'name': route_name,
            'coordinates': coordinates,
            'actual_time': milliseconds_to_hms(actual_time)  # 실제 소요 시간만 저장
        }
        routes_data.append(route_data)
    
    # JSON 파일로 저장
    with open(filename, 'w', encoding='utf-8') as file:
        json.dump(routes_data, file, ensure_ascii=False, indent=3)

In [19]:
def main():
    
    # 사용자로부터 구직자 위치 주소 입력 받기
    job_seeker_address = input("주소를 입력하세요 ex)서울시 송파구 올림픽로300 : ")
    job_seeker_location = get_coordinates_from_address(job_seeker_address)
    print(job_seeker_location)

    # 캐시 파일 초기화 (삭제)
    delete_cache_files(job_seeker_location)

    if job_seeker_location is None:
        print("주소의 좌표를 얻지 못했습니다. 프로그램을 종료합니다.")
        return
    
    print(f"구직자 위치 (위도, 경도): {job_seeker_location}: {job_seeker_location}")

    print("Loading data...")
    df = pd.read_csv('geocoded_stores2.csv', encoding='utf-8')
    seoul_stores = df[df['Road Address'].str.contains("서울")]
    
    print(f"서울의 매장 수: {len(seoul_stores)}")

    store_locations = seoul_stores[['shop_lat', 'shop_lng']].values
    store_names = seoul_stores['Name'].tolist()
    
    # 구직자 위치를 포함하여 최적 경로 찾기
    top_routes = find_top_routes(job_seeker_location, store_locations, store_names, num_routes=10)
    
    # 결과 출력
    print(f"Top routes from job seeker location at {job_seeker_location}:")
    for route_time, route in top_routes:
        route_names = ['Start'] + [store_names[i] for i in route] + ['Start']
        print(f"경로: {route_names} - 예상 시간: {milliseconds_to_hms(route_time)}")
        
        # 실제 경로 시간 측정
        actual_time = get_route_time(job_seeker_location, [store_locations[i] for i in route], job_seeker_location)
        print(f"실제 시간: {milliseconds_to_hms(actual_time)}")

    # 경로 데이터를 JSON 파일로 저장
    save_top_routes_to_file(top_routes, store_locations, store_names, job_seeker_location, 'top_routes.json')
if __name__ == "__main__":
    main()


(37.547076, 127.0516008)
Deleted cache file: cache\start_times_cache_-4384997961406863983.npy
Deleted cache file: cache\end_times_cache_-4384997961406863983.npy
구직자 위치 (위도, 경도): (37.547076, 127.0516008): (37.547076, 127.0516008)
Loading data...
서울의 매장 수: 63
Finding top routes for job seeker at (37.547076, 127.0516008)...
Number of stores: 63
Finding top routes for job seeker at (37.547076, 127.0516008)...
Calculating store-to-store distance matrix...
Attempting to load distance matrix from cache at cache\store_distance_matrix.npy...
Loaded distance matrix from cache.
Store-to-store distance matrix calculated:
[[      0. 1186325. 1596676. ... 2292082. 5539855. 2812705.]
 [1672234.       0. 2179050. ... 3353856. 5017666. 3911126.]
 [1319142. 1743630.       0. ... 2413359. 2655382. 3062294.]
 ...
 [2825812. 3241740. 3086461. ...       0. 3626351. 1096482.]
 [3047064. 2823614. 2545401. ... 2572511.       0. 2084009.]
 [3311699. 3729042. 3469761. ... 1171204. 2735562.       0.]]
Closest sto